In [1]:
import bz2
import shutil
%load_ext sql
%config SqlMagic.autopandas=True

dbfile = "doubutsu.db"
bz2file = "doubutsu.db.bz2"
tmpfile = "tmp.db"

In [2]:
# test original db file
%sql sqlite:///{dbfile}
a = %sql SELECT count(*) FROM stateValues
display(a)

a = %sql SELECT * FROM stateValues LIMIT 10
display(a)

 * sqlite:///doubutsu.db
Done.


,count(*)
0,109179572


 * sqlite:///doubutsu.db
Done.


,stateIndex,value
0,316013363792,-9900
1,316013393076,9801
2,316013685916,-9703
3,316013715200,9801
4,316022214942,-9703
5,316022241562,9801
6,316022243982,-9703
7,316022244202,-9900
8,316022244222,-9703
9,316022244226,-9703


In [5]:
%%time
# make bz2 db file
with open(dbfile, "rb") as f, bz2.open(bz2file, "wb") as g:
  shutil.copyfileobj(f, g)

CPU times: user 2min 1s, sys: 823 ms, total: 2min 2s
Wall time: 2min 3s


In [6]:
%%time
# decompress to a tmp file
with bz2.open(bz2file, "rb") as f, open(tmpfile, "wb") as g:
  shutil.copyfileobj(f, g)

CPU times: user 1min 7s, sys: 1.78 s, total: 1min 9s
Wall time: 1min 9s


In [7]:
# test the decompressed dbfile
%sql sqlite:///{tmpfile}
a = %sql SELECT count(*) FROM stateValues
display(a)

a = %sql SELECT * FROM stateValues LIMIT 10
display(a)

   sqlite:///doubutsu.db
 * sqlite:///tmp.db
Done.


,count(*)
0,109179572


   sqlite:///doubutsu.db
 * sqlite:///tmp.db
Done.


,stateIndex,value
0,316013363792,-9900
1,316013393076,9801
2,316013685916,-9703
3,316013715200,9801
4,316022214942,-9703
5,316022241562,9801
6,316022243982,-9703
7,316022244202,-9900
8,316022244222,-9703
9,316022244226,-9703


In [10]:
# calculate the md5 of the db file
import hashlib
def _get_md5(filename, blocksize=2**20):
    # compute md5 checksum of a file
    md5 = hashlib.md5()
    with open(filename, "rb") as f:
        while True:
            data = f.read(blocksize)
            if len(data) == 0:
                break
            md5.update(data)
    return md5.hexdigest()

%time md5 = _get_md5(dbfile)
print(md5)

# save to a text file
md5file = dbfile + ".md5"
with open(md5file, "w") as f:
    f.write(md5)

CPU times: user 2.89 s, sys: 284 ms, total: 3.17 s
Wall time: 3.17 s
95e0be3cf1dae308903fbfbf5223e3c3


In [12]:
# calculate the md5 of the db.bz2 file
%time md5 = _get_md5(bz2file)
print(md5)

# save to a text file
md5file = bz2file + ".md5"
with open(md5file, "w") as f:
    f.write(md5)

CPU times: user 906 ms, sys: 126 ms, total: 1.03 s
Wall time: 1.04 s
169258d9e9d53f8fe6d17d8c94845afb
